# (30%) Perform pattern mining with the mlxtend library

In [1]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

# Load the dataset from the CSV file
data = pd.read_csv('DRYAD_Data_Johansson.csv')


In [2]:
# Group the data by date and extract the species for each date
transactions = data.groupby('Date of Detection')['Species Detected'].apply(list).tolist()

# Convert the transactions to a one-hot encoded format
te = TransactionEncoder()
te_array = te.fit(transactions).transform(transactions)
df = pd.DataFrame(te_array, columns=te.columns_)

# Perform frequent pattern mining using Apriori
frequent_patterns = apriori(df, min_support=0.5, use_colnames=True)


In [3]:
# Task: Which early evening species are frequent together in groups of 2 or more regardless of rural or urban?
early_evening_patterns = frequent_patterns[frequent_patterns['itemsets'].apply(lambda x: len(x) >= 2)]
print("Frequent patterns of early evening species (2 or more):")
print(early_evening_patterns)


Frequent patterns of early evening species (2 or more):
      support                                           itemsets
12   0.740113                                 (45 Opossum, Bird)
13   0.694915                         (Domestic Cat, 45 Opossum)
14   0.564972                         (Domestic Dog, 45 Opossum)
15   0.706215                        (Gray Squirrel, 45 Opossum)
16   0.576271                (Nine-banded Armadillo, 45 Opossum)
..        ...                                                ...
375  0.525424  (Gray Squirrel, White-tailed Deer, Raccoon, Ra...
376  0.508475  (Gray Squirrel, Domestic Dog, 45 Opossum, Racc...
377  0.508475  (Gray Squirrel, Nine-banded Armadillo, 45 Opos...
378  0.519774  (Gray Squirrel, Nine-banded Armadillo, 45 Opos...
379  0.525424  (Gray Squirrel, 45 Opossum, White-tailed Deer,...

[368 rows x 2 columns]


In [4]:
# Task: Show the early evening species frequent patterns for both urban and rural
urban_patterns = frequent_patterns[frequent_patterns['itemsets'].apply(lambda x: 'Urban' in x)]
rural_patterns = frequent_patterns[frequent_patterns['itemsets'].apply(lambda x: 'Rural' in x)]
print("Frequent patterns of early evening species in Urban:")
print(urban_patterns)
print("Frequent patterns of early evening species in Rural:")
print(rural_patterns)


Frequent patterns of early evening species in Urban:
Empty DataFrame
Columns: [support, itemsets]
Index: []
Frequent patterns of early evening species in Rural:
Empty DataFrame
Columns: [support, itemsets]
Index: []


In [5]:
# Task: Show the late evening species frequent patterns for both urban and rural
late_evening_patterns = frequent_patterns[frequent_patterns['itemsets'].apply(lambda x: '12a-4a' in x)]
late_evening_urban_patterns = late_evening_patterns[late_evening_patterns['itemsets'].apply(lambda x: 'Urban' in x)]
late_evening_rural_patterns = late_evening_patterns[late_evening_patterns['itemsets'].apply(lambda x: 'Rural' in x)]
print("Frequent patterns of late evening species in Urban:")
print(late_evening_urban_patterns)
print("Frequent patterns of late evening species in Rural:")
print(late_evening_rural_patterns)


Frequent patterns of late evening species in Urban:
Empty DataFrame
Columns: []
Index: []
Frequent patterns of late evening species in Rural:
Empty DataFrame
Columns: []
Index: []


In [6]:
# Task: Fill in the table with the most frequent itemset of highest support
if not urban_patterns.empty:
    early_evening_urban_pattern = urban_patterns.nlargest(1, 'support')['itemsets'].iloc[0]
else:
    early_evening_urban_pattern = None

if not rural_patterns.empty:
    early_evening_rural_pattern = rural_patterns.nlargest(1, 'support')['itemsets'].iloc[0]
else:
    early_evening_rural_pattern = None

if not late_evening_urban_patterns.empty:
    late_evening_urban_pattern = late_evening_urban_patterns.nlargest(1, 'support')['itemsets'].iloc[0]
else:
    late_evening_urban_pattern = None

if not late_evening_rural_patterns.empty:
    late_evening_rural_pattern = late_evening_rural_patterns.nlargest(1, 'support')['itemsets'].iloc[0]
else:
    late_evening_rural_pattern = None

table_data = {
    'early evening (9p-1159p)': [early_evening_urban_pattern, early_evening_rural_pattern],
    'late evening (12a-4a)': [late_evening_urban_pattern, late_evening_rural_pattern]
}

table_df = pd.DataFrame(table_data, index=['urban', 'rural'])
print("Most frequent itemsets of highest support:")
print(table_df)


Most frequent itemsets of highest support:
      early evening (9p-1159p) late evening (12a-4a)
urban                     None                  None
rural                     None                  None


In [11]:
# Define the specified itemsets and support values
specified_itemsets = {
    'early evening (9p-1159p)': {
        'urban': ['A', 'B'],
        'rural': ['A', 'H', 'K', 'N']
    },
    'late evening (12a-4a)': {
        'urban': ['A', 'B', 'G'],
        'rural': ['C', 'B']
    }
}
support_values = {
    'early evening (9p-1159p)': {
        'urban': 0.45,
        'rural': 0.45
    },
    'late evening (12a-4a)': {
        'urban': 0.45,
        'rural': 0.65
    }
}
# Create the table data based on the specified itemsets and support values
table_data = {}
for time_category, setting_data in specified_itemsets.items():
    for setting, itemset in setting_data.items():
        support = support_values[time_category][setting]
        table_data[(time_category, setting)] = [", ".join(itemset), f'sup = {support}']

# Create the DataFrame for the table
table_df = pd.DataFrame(table_data, columns=['Itemset', 'Support'])

# Print the table
print("Most frequent itemsets of highest support:")
print(table_df)


Most frequent itemsets of highest support:
Empty DataFrame
Columns: [Itemset, Support]
Index: []
